# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `production_2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import dask
dask.config.set({'dataframe.query-planning': True})
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [16]:
import os
from glob import glob

price_data_dir = os.getenv("PRICE_DATA")
print(price_data_dir)

parquet_files = glob(price_data_dir + "/**/*.parquet", recursive=True)


../../05_src/data/prices/


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [21]:
import os
from glob import glob
import shutil

# Clean up the folder structure
# Move any folders with ".parquet" in their names to a temporary location
temp_dir = os.path.join(price_data_dir, 'temp')
os.makedirs(temp_dir, exist_ok=True)

for root, dirs, files in os.walk(price_data_dir):
    for dir_name in dirs:
        if dir_name.endswith('.parquet'):
            # Define the destination path
            dst = os.path.join(temp_dir, dir_name)
            counter = 1
            while os.path.exists(dst):
                dst = os.path.join(temp_dir, f"{dir_name}_{counter}")
                counter += 1
            shutil.move(os.path.join(root, dir_name), dst)

# Find all actual parquet files in the cleaned directory structure
parquet_files = glob(os.path.join(price_data_dir, "**/*.parquet"), recursive=True)
print(parquet_files)

['../../05_src/data/prices\\temp\\AAL_2005.parquet_1\\part.0.parquet', '../../05_src/data/prices\\temp\\AAL_2006.parquet_1\\part.0.parquet', '../../05_src/data/prices\\temp\\AAL_2007.parquet_1\\part.0.parquet', '../../05_src/data/prices\\temp\\AAL_2008.parquet_1\\part.0.parquet', '../../05_src/data/prices\\temp\\AAL_2009.parquet_1\\part.0.parquet', '../../05_src/data/prices\\temp\\AAL_2010.parquet_1\\part.0.parquet', '../../05_src/data/prices\\temp\\AAL_2011.parquet_1\\part.0.parquet', '../../05_src/data/prices\\temp\\AAL_2012.parquet_1\\part.0.parquet', '../../05_src/data/prices\\temp\\AAL_2013.parquet_1\\part.0.parquet', '../../05_src/data/prices\\temp\\AAL_2014.parquet_1\\part.0.parquet', '../../05_src/data/prices\\temp\\AAL_2015.parquet_1\\part.0.parquet', '../../05_src/data/prices\\temp\\AAL_2016.parquet_1\\part.0.parquet', '../../05_src/data/prices\\temp\\AAL_2017.parquet_1\\part.0.parquet', '../../05_src/data/prices\\temp\\AAL_2018.parquet_1\\part.0.parquet', '../../05_src/data/

In [28]:
# Write your code below.
import dask.dataframe as dd

# Load parquet files
ddf = dd.read_parquet(parquet_files)
print(ddf.columns)
print(ddf.head())

# Add features
def add_features(df):
    # Add lags, returns and high/low range for variable Close not Adj_close as instructed in slack by Jesus
    df['Close_lag'] = df['Close'].shift(1)
    
    df['returns'] = (df['Close'] / df['Close_lag']) - 1
    
    df['hi_lo_range'] = df['High'] - df['Low']
    
    return df

# Update the metadata to include the new columns
meta = ddf.head()
meta['Close_lag'] = meta['Close']
meta['returns'] = meta['Close']
meta['hi_lo_range'] = meta['High']

# Apply the function to add features
dd_feat = ddf.groupby('ticker').apply(add_features, meta=meta)


Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'sector',
       'subsector', 'year'],
      dtype='object')
             Date       Open       High        Low      Close  Adj Close  \
ticker                                                                     
AAL    2005-09-27  21.049999  21.400000  19.100000  19.299999  18.194912   
AAL    2005-09-28  19.299999  20.530001  19.200001  20.500000  19.326206   
AAL    2005-09-29  20.400000  20.580000  20.100000  20.209999  19.052803   
AAL    2005-09-30  20.260000  21.049999  20.180000  21.010000  19.806999   
AAL    2005-10-03  20.900000  21.750000  20.900000  21.500000  20.268940   

         Volume       sector           subsector  year  
ticker                                                  
AAL      961200  Industrials  Passenger Airlines  2005  
AAL     5747900  Industrials  Passenger Airlines  2005  
AAL     1078200  Industrials  Passenger Airlines  2005  
AAL     3123300  Industrials  Passenger Airlines  2

+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [27]:
# Write your code below.
# Convert the Dask data frame to pandas data frame
pdf = dd_feat.compute()

pdf['rolling_avg_return'] = pdf['returns'].rolling(10).mean()
print(pdf.head())


                    Date       Open       High        Low      Close  \
ticker ticker                                                          
OXY    OXY    2012-01-03  92.673279  93.765472  91.418221  92.577477   
       OXY    2012-01-04  92.395447  93.267281  91.676895  92.855316   
       OXY    2012-01-05  92.721184  93.123573  91.322418  92.117607   
       OXY    2012-01-06  92.874474  92.970284  90.814644  91.753540   
       OXY    2012-01-09  91.638573  93.343925  91.609833  92.864891   

               Adj Close   Volume  sector                           subsector  \
ticker ticker                                                                   
OXY    OXY     63.047344  4539584  Energy  Oil & Gas Exploration & Production   
       OXY     63.236588  3922191  Energy  Oil & Gas Exploration & Production   
       OXY     62.734165  3667093  Energy  Oil & Gas Exploration & Production   
       OXY     62.486233  3185704  Energy  Oil & Gas Exploration & Production   
       OX

Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

1. There is no need to convert to pandas as Dask also supports rolling windows calculations by rolling() method.

2. The larger the dataset, the better it is to use Dask because it proceses data in parallel. Pandas work in the same memory/machine and hence is not that efficient. Dask is more scalable and efficient.

## Criteria

|Criteria|Complete|Incomplete|
|---------------------|----|----|
|Calculations         |Calculations were done correctly.|Calculations were not done correctly.|
|Explanation of answer|Answer was concise and explained the learner's reasoning in depth.|Answer was not concise and did not explained the learner's reasoning in depth.|

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.